# Untyped Lambda Calculus

In [1]:
from copy import deepcopy
from nanoid import generate


In [2]:
def genid(n=6):
    s = "abcdefghijklmnopqrstuvwxyz" + "abcdefghijklmnopqrstuvwxyz".upper()
    return generate(s, n)


In [3]:
genid()


'TutxuY'

In [4]:
class parser:
    """
    # S-Exp Parser
        Grammar:
            P -> id | E
            L -> P | P L
            E -> (L)

        LL(1):
            P  -> id | E
            L  -> PL'
            L' -> L | null
            E  -> (L)
    """

    def lexer(s):
        s = s.replace("\n", " ").replace("(", " ( ").replace(")", " ) ")
        l = s.split()
        ret = []
        for c in l:
            if c == "(":
                ret.append(("(", None))
            elif c == ")":
                ret.append((")", None))
            else:
                ret.append(("id", c))
        return ret

    def __init__(self, s):
        self.tokens = list(reversed(parser.lexer(s)))

    def run(self):
        return self.E()

    def next(self):
        self.tokens.pop()

    def peek(self):
        return self.tokens[-1]

    def P(self):
        token = self.peek()
        if token[0] == "id":
            # print("P id={}".format(token[1]))
            self.next()
            return token[1]
        else:
            ret = self.E()
            return ret

    def L(self):
        token = self.peek()
        ret = [self.P()]
        ret += self.Lp()
        return ret

    def Lp(self):
        token = self.peek()
        if token[0] == "id" or token[0] == "(":
            ret = self.L()
            return ret
        else:
            return []

    def E(self):
        token = self.peek()
        assert token[0] == "("
        self.next()
        ret = self.L()
        token = self.peek()
        assert token[0] == ")"
        self.next()
        return ret


# NbE

In [6]:
# Normal Forms of Untyped λ-Calculus
#  ‹expr› ::= ‹id›
#          |  ( λ ( ‹id› ) ‹expr› )
#          |  ( ‹expr› ‹expr› )
# LL1:
# ‹norm›::= ‹neu›|( λ ( ‹id› ) ‹norm› )
# ‹neu›::=‹id›|( ‹neu› ‹norm› )


In [7]:
class nbe_runner:
    def __init__(self, slist):
        self.prog = slist
        self.indent = 0

    def clos(self, env, var, body):
        return ["CLOS", env, var, body]

    def assv(self, name, env):
        try:
            return env[name]
        except:
            return False

    def extend(self, env, name, val):
        ret = deepcopy(env)
        ret[name] = val
        return ret

    def is_symbol(self, s):
        if type(s) == str:
            return True
        return False

    def freshen(self, used, x):
        # print("FRESHEN", str(used), x)
        if x in used:
            return self.freshen(used, x + "*")
        else:
            return x

    def lazy(self, env, exp):
        return ["LAZY", env, exp]

    def n_var(self, x):
        return ["N-var", x, genid()]

    def n_ap(self, rator, rand):
        return ["N-ap", rator, rand]

    def val(self, env, exp):
        # print("  " * self.indent, end="")
        # print("EVAL", exp, "in", env)
        self.indent += 1

        ret = None
        match exp:
            case ["λ", [var], body]:
                ret = self.clos(env, var, body)
            # case ["LAZY", env, exp]:
            #     return self.val(env, exp)
            case [rator, rand]:
                ret = self.do_ap(self.val(env, rator), self.val(env, rand))
            case x:
                xv = self.assv(x, env)
                if xv != False:
                    ret = xv
                else:
                    # raise RuntimeError("Unknown variable {}".format(exp))
                    return self.n_var(x)
        self.indent -= 1
        return ret

    def do_ap(self, fun, arg):
        # print("  " * self.indent, end="")
        # print("APPLY", fun, "to", arg)
        self.indent += 1

        ret = None
        match fun:
            case ["CLOS", env, var, body]:
                nenv = self.extend(env, var, arg)
                # ret = self.val(nenv, body)
                ret = self.lazy(nenv, body)
            case ["LAZY", env, exp]:
                f = self.val(env, exp)
                ret = self.do_ap(f, arg)
            case _:
                ret = self.n_ap(fun, arg)
        self.indent -= 1
        return ret

    def read_back(self, used_names, v):
        # print("  " * self.indent, end="")
        # print("READBACK", used_names, v)
        self.indent += 1
        ret = None
        match v:
            case ["CLOS", env, x, body]:
                y = self.freshen(used_names, x)
                ny = self.n_var(y)
                ret = [
                    "λ",
                    [y],
                    self.read_back(
                        used_names + [y], self.val(self.extend(env, x, ny), body)
                    ),
                ]
            case ["LAZY", env, exp]:
                # raise RuntimeError
                x = self.val(env, exp)
                ret = self.read_back(used_names, x)
            case ["N-var", var, gid]:
                ret = var+'_'+gid
                # ret = var
            case ["N-ap", rator, rand]:
                ret = [
                    self.read_back(used_names, rator),
                    self.read_back(used_names, rand),
                ]
            case _:
                raise RuntimeError("Read Back Failed")
        self.indent -= 1
        return ret

    def norm(self, env, exp):
        # print("  " * self.indent, end="")
        # print("NORM", str(exp))

        self.indent += 1
        ret = self.read_back([], self.val(env, exp))
        self.indent -= 1
        return ret

    def _run(self, env, exprs):
        self.loc += 1
        if len(exprs) > 0:
            exp = exprs[0]
            rest = exprs[1:]
            if exp[0] == "define":
                x = exp[1]
                e = exp[2]
                v = self.val(env, e)
                self._run(self.extend(env, x, v), rest)
            else:
                print("#", self.loc, ">", self.norm(env, exp))
                self._run(env, rest)
        return

    def run(self):
        self.loc = 0
        self._run({}, self.prog)


class nbe_interpretor:
    def __init__(self, prog):
        p = parser(prog)
        slist = p.run()
        self.exec = nbe_runner(slist)

    def run(self):
        self.exec.run()


In [15]:
class nbe_runner:
    def __init__(self, slist):
        self.prog = slist
        self.indent = 0

    def clos(self, env, var, body):
        return ["CLOS", env, var, body]

    def assv(self, name, env):
        try:
            return env[name]
        except:
            return False

    def extend(self, env, name, val):
        ret = deepcopy(env)
        ret[name] = val
        return ret

    def is_symbol(self, s):
        if type(s) == str:
            return True
        return False

    def freshen(self, used, x):
        # print("FRESHEN", str(used), x)
        if x in used:
            return self.freshen(used, x + "*")
        else:
            return x

    def lazy(self, env, exp):
        return ["LAZY", env, exp]

    def n_var(self, x):
        return ["N-var", x]

    def n_ap(self, rator, rand):
        return ["N-ap", rator, rand]

    def val(self, env, exp):
        # print("  " * self.indent, end="")
        # print("EVAL", exp, "in", env)
        self.indent += 1

        ret = None
        match exp:
            case ["λ", [var], body]:
                ret = self.clos(env, var, body)
            # case ["LAZY", env, exp]:
            #     return self.val(env, exp)
            case [rator, rand]:
                ret = self.do_ap(self.val(env, rator), self.val(env, rand))
            case x:
                xv = self.assv(x, env)
                if xv != False:
                    ret = xv
                else:
                    # raise RuntimeError("Unknown variable {}".format(exp))
                    return self.n_var(x)
        self.indent -= 1
        return ret

    def do_ap(self, fun, arg):
        # print("  " * self.indent, end="")
        # print("APPLY", fun, "to", arg)
        self.indent += 1

        ret = None
        match fun:
            case ["CLOS", env, var, body]:
                nenv = self.extend(env, var, arg)
                # ret = self.val(nenv, body)
                ret = self.lazy(nenv, body)
            case ["LAZY", env, exp]:
                f = self.val(env, exp)
                ret = self.do_ap(f, arg)
            case _:
                ret = self.n_ap(fun, arg)
        self.indent -= 1
        return ret

    def read_back(self, used_names, v):
        # print("  " * self.indent, end="")
        # print("READBACK", used_names, v)
        self.indent += 1
        ret = None
        match v:
            case ["CLOS", env, x, body]:
                y = self.freshen(used_names, x)
                ny = self.n_var(y)
                ret = [
                    "λ",
                    [y],
                    self.read_back(
                        used_names + [y], self.val(self.extend(env, x, ny), body)
                    ),
                ]
            case ["LAZY", env, exp]:
                # raise RuntimeError
                x = self.val(env, exp)
                ret = self.read_back(used_names, x)
            case ["N-var", var]:
                ret = var
                # ret = var
            case ["N-ap", rator, rand]:
                ret = [
                    self.read_back(used_names, rator),
                    self.read_back(used_names, rand),
                ]
            case _:
                raise RuntimeError("Read Back Failed")
        self.indent -= 1
        return ret

    def norm(self, env, exp):
        # print("  " * self.indent, end="")
        # print("NORM", str(exp))

        self.indent += 1
        ret = self.read_back([], self.val(env, exp))
        self.indent -= 1
        return ret

    def _run(self, env, exprs):
        self.loc += 1
        if len(exprs) > 0:
            exp = exprs[0]
            rest = exprs[1:]
            if exp[0] == "define":
                x = exp[1]
                e = exp[2]
                v = self.val(env, e)
                self._run(self.extend(env, x, v), rest)
            else:
                print("#", self.loc, ">", self.norm(env, exp))
                self._run(env, rest)
        return

    def run(self):
        self.loc = 0
        self._run({}, self.prog)


class nbe_interpretor:
    def __init__(self, prog):
        p = parser(prog)
        slist = p.run()
        self.exec = nbe_runner(slist)

    def run(self):
        self.exec.run()


In [16]:
prog04 = """
(
    (
        (λ (x) y) 
        (
            (λ (x) (x x))
            (λ (x) (x x))
        )
    )
)
"""
test = nbe_interpretor(prog04)
test.run()


# 1 > y


In [17]:
prog04 = """
(
    (
        (λ (x) ((λ (x) x) x)) x
    )
)
"""
test = nbe_interpretor(prog04)
test.run()


# 1 > x


In [18]:
testrb = nbe_runner([])
testrb.norm({}, [["λ", ["x"], ["λ", ["y"], ["x", "y"]]], ["λ", ["x"], "x"]])


['λ', ['y'], 'y']

In [11]:
prog02 = """
((define z
       (λ (f)
         (λ (x)
           x)))
     (define s
       (λ (n)
         (λ (f)
           (λ (x)
             (f ((n f) x))))))
     (s (s z))))
"""
test = nbe_interpretor(prog02)
test.run()


# 3 > ['λ', ['f'], ['λ', ['x'], ['f_YVWIOy', ['f_YVWIOy', 'x_TGHHKQ']]]]


# NbE Play

In [12]:
def with_numerals(exp):
    s = (
        """((define ZERO
      (λ (f)
        (λ (x)
          x)))
    (define ADD1
      (λ (n-1)
        (λ (f)
          (λ (x)
            (f ((n-1 f) x))))))
    """
        + exp
        + ")"
    )
    ret = parser(s).run()
    # ret = s
    return ret


def to_church(n):
    if n <= 0:
        return "ZERO"
    else:
        return "(ADD1 {})".format(to_church(n - 1))


def church_add(i, j):
    return """
    (((λ (j)
     (λ (k)
       (λ (f)
         (λ (x)
           ((j f) ((k f) x)))))) {}) {})
    """.format(
        i, j
    )


In [13]:
prog02 = with_numerals(church_add(to_church(2), to_church(3)))
for i, s in enumerate(prog02):
    print(i + 1, s)
print()
testrb = nbe_runner(prog02)
testrb.run()


1 ['define', 'ZERO', ['λ', ['f'], ['λ', ['x'], 'x']]]
2 ['define', 'ADD1', ['λ', ['n-1'], ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]]]
3 [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', ['ADD1', 'ZERO']]], ['ADD1', ['ADD1', ['ADD1', 'ZERO']]]]

# 3 > ['λ', ['f'], ['λ', ['x'], ['f_erSDbn', ['f_erSDbn', ['f_erSDbn', ['f_erSDbn', ['f_erSDbn', 'x_wLPHMW']]]]]]]


In [19]:
prog03 = """
(
    (((λ (f) (λ (x) (f x))) x) z)
)
"""
test = nbe_interpretor(prog03)
test.run()


# 1 > ['x', 'z']


In [52]:
prog04 = """
(
    (
        (λ (x) y) 
        (
            (λ (x) (x x))
            (λ (x) (x x))
        )
    )
)
"""
test = nbe_interpretor(prog04)
test.run()


# 1 > y


In [53]:
prog02 = with_numerals(church_add(to_church(1), to_church(1)))
for i, s in enumerate(prog02):
    print(i + 1, s)
print()
testrb = nbe_runner(prog02)
testrb.run()

1 ['define', 'ZERO', ['λ', ['f'], ['λ', ['x'], 'x']]]
2 ['define', 'ADD1', ['λ', ['n-1'], ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]]]
3 [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', 'ZERO']], ['ADD1', 'ZERO']]

# 3 > ['λ', ['f'], ['λ', ['x'], ['f', ['f', 'x']]]]


In [6]:
# Fuzz
from random import choice, random

In [7]:
def genlmd(var, exp):
    return ["λ", [var], exp]


def genap(f, v):
    return [f, v]


def genexp(vars, pl, pe):
    r = random()
    ret = None
    if r < pl:
        ret = genlmd(choice(vars), genexp(vars, pl, pe))
    elif r < pl + pe:
        ret = genap(genexp(vars, pl, pe), genexp(vars, pl, pe))
    else:
        ret = choice(vars)
    return ret


In [5]:
class nbe_runner:
    def __init__(self, slist):
        self.prog = slist
        self.indent = 0

    def clos(self, env, var, body):
        return ["CLOS", env, var, body]

    def assv(self, name, env):
        try:
            return env[name]
        except:
            return False

    def extend(self, env, name, val):
        ret = deepcopy(env)
        ret[name] = val
        return ret

    def is_symbol(self, s):
        if type(s) == str:
            return True
        return False

    def freshen(self, used, x):
        # print("FRESHEN", str(used), x)
        if x in used:
            return self.freshen(used, x + "*")
        else:
            return x

    def lazy(self, env, exp):
        return ["LAZY", env, exp]

    def n_var(self, x):
        return ["N-var", x, genid()]

    def n_ap(self, rator, rand):
        return ["N-ap", rator, rand]

    def val(self, env, exp):
        # print("  " * self.indent, end="")
        # print("EVAL", exp, "in", env)
        self.indent += 1

        ret = None
        match exp:
            case ["λ", [var], body]:
                ret = self.clos(env, var, body)
            # case ["LAZY", env, exp]:
            #     return self.val(env, exp)
            case [rator, rand]:
                ret = self.do_ap(self.val(env, rator), self.val(env, rand))
            case x:
                xv = self.assv(x, env)
                if xv != False:
                    ret = xv
                else:
                    # raise RuntimeError("Unknown variable {}".format(exp))
                    return self.n_var(x)
        self.indent -= 1
        return ret

    def do_ap(self, fun, arg):
        # print("  " * self.indent, end="")
        # print("APPLY", fun, "to", arg)
        self.indent += 1

        ret = None
        match fun:
            case ["CLOS", env, var, body]:
                nenv = self.extend(env, var, arg)
                # ret = self.val(nenv, body)
                ret = self.lazy(nenv, body)
            case ["LAZY", env, exp]:
                f = self.val(env, exp)
                ret = self.do_ap(f, arg)
            case _:
                ret = self.n_ap(fun, arg)
        self.indent -= 1
        return ret

    def read_back(self, used_names, v):
        # print("  " * self.indent, end="")
        # print("READBACK", used_names, v)
        self.indent += 1
        ret = None
        match v:
            case ["CLOS", env, x, body]:
                y = self.freshen(used_names, x)
                ny = self.n_var(y)
                ret = [
                    "λ",
                    [y],
                    self.read_back(
                        used_names + [y], self.val(self.extend(env, x, ny), body)
                    ),
                ]
            case ["LAZY", env, exp]:
                # raise RuntimeError
                x = self.val(env, exp)
                ret = self.read_back(used_names, x)
            case ["N-var", var, gid]:
                ret = var+'_'+gid
                # ret = var
            case ["N-ap", rator, rand]:
                ret = [
                    self.read_back(used_names, rator),
                    self.read_back(used_names, rand),
                ]
            case _:
                raise RuntimeError("Read Back Failed")
        self.indent -= 1
        return ret

    def norm(self, env, exp):
        # print("  " * self.indent, end="")
        # print("NORM", str(exp))

        self.indent += 1
        ret = self.read_back([], self.val(env, exp))
        self.indent -= 1
        return ret

    def _run(self, env, exprs):
        self.loc += 1
        ret = ""
        if len(exprs) > 0:
            exp = exprs[0]
            rest = exprs[1:]
            if exp[0] == "define":
                x = exp[1]
                e = exp[2]
                v = self.val(env, e)
                self._run(self.extend(env, x, v), rest)
            else:
                print("#", self.loc, ">", self.norm(env, exp))
                self._run(env, rest)
        return

    def run(self):
        self.loc = 0
        self._run({}, self.prog)


class nbe_interpretor:
    def __init__(self, prog):
        p = parser(prog)
        slist = p.run()
        self.exec = nbe_runner(slist)

    def run(self):
        self.exec.run()


In [17]:
class nbe_runner_nfr:
    def __init__(self, slist):
        self.prog = slist
        self.indent = 0

    def clos(self, env, var, body):
        return ["CLOS", env, var, body]

    def assv(self, name, env):
        try:
            return env[name]
        except:
            return False

    def extend(self, env, name, val):
        ret = deepcopy(env)
        ret[name] = val
        return ret

    def is_symbol(self, s):
        if type(s) == str:
            return True
        return False

    def freshen(self, used, x):
        # print("FRESHEN", str(used), x)
        # if x in used:
        #     return self.freshen(used, x + "*")
        # else:
        #     return x
        return x

    def lazy(self, env, exp):
        return ["LAZY", env, exp]

    def n_var(self, x):
        return ["N-var", x]

    def n_ap(self, rator, rand):
        return ["N-ap", rator, rand]

    def val(self, env, exp):
        print("  " * self.indent, end="")
        print("EVAL", exp, "in", env)
        self.indent += 1

        ret = None
        match exp:
            case ["λ", [var], body]:
                ret = self.clos(env, var, body)
            # case ["LAZY", env, exp]:
            #     return self.val(env, exp)
            case [rator, rand]:
                ret = self.do_ap(self.val(env, rator), self.val(env, rand))
            case x:
                xv = self.assv(x, env)
                if xv != False:
                    ret = xv
                else:
                    # raise RuntimeError("Unknown variable {}".format(exp))
                    ret = self.n_var(x)
        self.indent -= 1
        return ret

    def do_ap(self, fun, arg):
        print("  " * self.indent, end="")
        print("APPLY", fun, "to", arg)
        self.indent += 1

        ret = None
        match fun:
            case ["CLOS", env, var, body]:
                nenv = self.extend(env, var, arg)
                ret = self.val(nenv, body)
                # ret = self.lazy(nenv, body)
            case ["LAZY", env, exp]:
                f = self.val(env, exp)
                ret = self.do_ap(f, arg)
            case _:
                ret = self.n_ap(fun, arg)
        self.indent -= 1
        return ret

    def read_back(self, used_names, v):
        print("  " * self.indent, end="")
        # print("READBACK", used_names, v)
        print("READBACK", v)
        self.indent += 1
        ret = None
        match v:
            case ["CLOS", env, x, body]:
                y = self.freshen(used_names, x)
                ny = self.n_var(y)
                ret = [
                    "λ",
                    [y],
                    self.read_back(
                        used_names + [y], self.val(self.extend(env, x, ny), body)
                    ),
                ]
            case ["LAZY", env, exp]:
                # raise RuntimeError
                x = self.val(env, exp)
                ret = self.read_back(used_names, x)
            case ["N-var", var]:
                ret = var
                # ret = var
            case ["N-ap", rator, rand]:
                ret = [
                    self.read_back(used_names, rator),
                    self.read_back(used_names, rand),
                ]
            case _:
                raise RuntimeError("Read Back Failed")
        self.indent -= 1
        return ret

    def norm(self, env, exp):
        print("  " * self.indent, end="")
        print("NORM", str(exp))

        self.indent += 1
        ret = self.read_back([], self.val(env, exp))
        self.indent -= 1
        return ret

    def _run(self, env, exprs):
        self.loc += 1
        if len(exprs) > 0:
            exp = exprs[0]
            rest = exprs[1:]
            if exp[0] == "define":
                x = exp[1]
                e = exp[2]
                v = self.val(env, e)
                self._run(self.extend(env, x, v), rest)
                # print("#", self.loc, ">", v)
            else:
                print("#", self.loc, ">", self.norm(env, exp))
                self._run(env, rest)
        return

    def run(self):
        self.loc = 0
        self._run({}, self.prog)

class nbe_interpretor_nfr:
    def __init__(self, prog):
        p = parser(prog)
        slist = p.run()
        self.exec = nbe_runner_nfr(slist)

    def run(self):
        self.exec.run()


In [14]:
import re
p = re.compile("_[a-zA-Z]{6,6}")
print(p.sub("", "x_gWItoO, x_gWItoO"))

x, x


In [7]:
p = re.compile("_[a-zA-Z]{6,6}")
for i in range(10000):
    try:
        exp = genexp(['f', 'x'], 0.3, 0.2)
        if len(str(exp)) < 100:
            continue
        else:
            testrb1 = nbe_runner([])
            ret1 = str(testrb1.norm({}, exp))
            ret1 = ret1.replace("*", "")
            ret1 = p.sub("", ret1)
            testrb2 = nbe_runner_nfr([])
            ret2 = str(testrb2.norm({}, exp))
            ret2 = p.sub("", ret2)
            # print(str(exp))
            # print(ret1)
            # print(ret2)
            # print()
            assert(ret1 == ret2)
    except Exception as e:
        print(exp)
        print(e)
        print()

NameError: name 'exp' is not defined

In [8]:
prog03 = """
(
    ((λ (f) (λ (x) (f x))) x)
)
"""
test = nbe_interpretor_nfr(prog03)
test.run()

NORM [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x']
  EVAL [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'] in {}
    EVAL ['λ', ['f'], ['λ', ['x'], ['f', 'x']]] in {}
    EVAL x in {}
    APPLY ['CLOS', {}, 'f', ['λ', ['x'], ['f', 'x']]] to ['N-var', 'x']
      EVAL ['λ', ['x'], ['f', 'x']] in {'f': ['N-var', 'x']}
  READBACK ['CLOS', {'f': ['N-var', 'x']}, 'x', ['f', 'x']]
    EVAL ['f', 'x'] in {'f': ['N-var', 'x'], 'x': ['N-var', 'x']}
      EVAL f in {'f': ['N-var', 'x'], 'x': ['N-var', 'x']}
      EVAL x in {'f': ['N-var', 'x'], 'x': ['N-var', 'x']}
      APPLY ['N-var', 'x'] to ['N-var', 'x']
    READBACK ['N-ap', ['N-var', 'x'], ['N-var', 'x']]
      READBACK ['N-var', 'x']
      READBACK ['N-var', 'x']
# 1 > ['λ', ['x'], ['x', 'x']]


In [20]:
prog03 = """
(
    (define ff ((λ (f) (λ (x) (f x))) x))
    (ff a)
)
"""
test = nbe_interpretor_nfr(prog03)
test.run()


EVAL [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'] in {}
  EVAL ['λ', ['f'], ['λ', ['x'], ['f', 'x']]] in {}
  EVAL x in {}
  APPLY ['CLOS', {}, 'f', ['λ', ['x'], ['f', 'x']]] to ['N-var', 'x']
    EVAL ['λ', ['x'], ['f', 'x']] in {'f': ['N-var', 'x']}
NORM ['ff', 'a']
  EVAL ['ff', 'a'] in {'ff': ['CLOS', {'f': ['N-var', 'x']}, 'x', ['f', 'x']]}
    EVAL ff in {'ff': ['CLOS', {'f': ['N-var', 'x']}, 'x', ['f', 'x']]}
    EVAL a in {'ff': ['CLOS', {'f': ['N-var', 'x']}, 'x', ['f', 'x']]}
    APPLY ['CLOS', {'f': ['N-var', 'x']}, 'x', ['f', 'x']] to ['N-var', 'a']
      EVAL ['f', 'x'] in {'f': ['N-var', 'x'], 'x': ['N-var', 'a']}
        EVAL f in {'f': ['N-var', 'x'], 'x': ['N-var', 'a']}
        EVAL x in {'f': ['N-var', 'x'], 'x': ['N-var', 'a']}
        APPLY ['N-var', 'x'] to ['N-var', 'a']
  READBACK ['N-ap', ['N-var', 'x'], ['N-var', 'a']]
    READBACK ['N-var', 'x']
    READBACK ['N-var', 'a']
# 2 > ['x', 'a']


In [19]:
prog03 = """
(
    (((λ (f) (λ (x) (f x))) x) a)
)
"""
test = nbe_interpretor_nfr(prog03)
test.run()

NORM [[['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'], 'a']
  EVAL [[['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'], 'a'] in {}
    EVAL [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'] in {}
      EVAL ['λ', ['f'], ['λ', ['x'], ['f', 'x']]] in {}
      EVAL x in {}
      APPLY ['CLOS', {}, 'f', ['λ', ['x'], ['f', 'x']]] to ['N-var', 'x']
        EVAL ['λ', ['x'], ['f', 'x']] in {'f': ['N-var', 'x']}
    EVAL a in {}
    APPLY ['CLOS', {'f': ['N-var', 'x']}, 'x', ['f', 'x']] to ['N-var', 'a']
      EVAL ['f', 'x'] in {'f': ['N-var', 'x'], 'x': ['N-var', 'a']}
        EVAL f in {'f': ['N-var', 'x'], 'x': ['N-var', 'a']}
        EVAL x in {'f': ['N-var', 'x'], 'x': ['N-var', 'a']}
        APPLY ['N-var', 'x'] to ['N-var', 'a']
  READBACK ['N-ap', ['N-var', 'x'], ['N-var', 'a']]
    READBACK ['N-var', 'x']
    READBACK ['N-var', 'a']
# 1 > ['x', 'a']


In [10]:
prog03 = """
(
    (((λ (z) 
         ((λ (f) (λ (x) (f x))) x)) 
         a) 
    b)
)
"""
test = nbe_interpretor_nfr(prog03)
test.run()


NORM [[['λ', ['z'], [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x']], 'a'], 'b']
  EVAL [[['λ', ['z'], [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x']], 'a'], 'b'] in {}
    EVAL [['λ', ['z'], [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x']], 'a'] in {}
      EVAL ['λ', ['z'], [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x']] in {}
      EVAL a in {}
      APPLY ['CLOS', {}, 'z', [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x']] to ['N-var', 'a']
        EVAL [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'] in {'z': ['N-var', 'a']}
          EVAL ['λ', ['f'], ['λ', ['x'], ['f', 'x']]] in {'z': ['N-var', 'a']}
          EVAL x in {'z': ['N-var', 'a']}
          APPLY ['CLOS', {'z': ['N-var', 'a']}, 'f', ['λ', ['x'], ['f', 'x']]] to ['N-var', 'x']
            EVAL ['λ', ['x'], ['f', 'x']] in {'z': ['N-var', 'a'], 'f': ['N-var', 'x']}
    EVAL b in {}
    APPLY ['CLOS', {'z': ['N-var', 'a'], 'f': ['N-var', 'x']}, 'x', ['f', 'x']] to ['N-var', 'b']
      EVAL ['f', 'x'] in {'z': ['N-var', 'a'], 'f': ['N-

In [26]:
testrun = nbe_runner_nfr([
    [
        "λ",
        ["f"],
        [
            "f",
            [
                "f",
                [
                    ["λ", ["x"], [[[["λ", ["x"], "x"], "f"], "x"], ["λ", ["f"], "f"]]],
                    "f",
                ],
            ],
        ],
    ]]
)

testrun.run()


# 1 > ['λ', ['f'], ['f', ['f', [['f', 'f'], ['λ', ['f'], 'f']]]]]
